<a href="https://colab.research.google.com/github/dxiong2001/malibu-ml/blob/main/html_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#imports 
from bs4 import BeautifulSoup
import requests
import nltk
from nltk.tokenize import sent_tokenize
import re
import unicodedata
import spacy

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [9]:
def get_html(url):
  page = requests.get(url)
  return BeautifulSoup(page.content, 'html.parser')


def getArticleTextSections(page_content):
  body_text = page_content.find('div', class_='Article-bodyText')
  body_text_p = body_text.find_all('p', class_='')

  processed_text1 = [item.get_text() for item in body_text_p]
  processed_text2=[]
  for processed in processed_text1:
    p2 = processed.replace('“', '"').replace('”', '"').replace("‘", "'").replace("’", "'").replace('…', '...').replace('–', '-')
    processed_text2.append(unicodedata.normalize('NFKD', p2))
  
  return processed_text2
  

def getCombinedText(processed_text):
  sentences = [sent_tokenize(sent) for sent in processed_text]

  combined_sentences = []
  for sent in sentences:
    combined_sentences += sent

  return combined_sentences


def getQuotes(processed_text):
  combined_sentences = getCombinedText(processed_text)

  quotes_incomplete = [re.findall('"([^"]*)"', paragraph) for paragraph in processed_text]
  quotes_incomplete = [q for q in quotes_incomplete if q != []]
  quotes_incomplete2 = [j for i in quotes_incomplete for j in i]

  quotes_complete =[]

  i=0
  for comb in combined_sentences:
    
    if(quotes_incomplete2[i] in comb):
      if(comb not in quotes_complete):
        quotes_complete.append(comb)
      i = i + 1
      while i < len(quotes_incomplete2) and quotes_incomplete2[i] in comb:
          i = i + 1
      
    
    if(i==len(quotes_incomplete2)):
        break

  return quotes_complete

def getArticleInfo(page_content):

  #grab article information
  header = page_content.find('div', class_='Article-header')
  article_info = header.find('p', class_='Article-author').get_text()
  title = header.find('h1', class_='u-entryTitle').get_text()
  author = article_info.split("|")[0].replace("\n                    ","").replace("\n              ","")[2:]
  date = article_info.split("|")[1].replace("\n    \n    Updated ","").replace("\n  \n","")

  return title, author, date




def getNamedEntities(processed_text):
  joined_sentences = ' '.join(getCombinedText(processed_text))
  print(joined_sentences)
  nlp_processed_text = nlp(joined_sentences)

  named_entities=[]
  for entities in nlp_processed_text.ents:
    named_entities.append((entities.text, entities.label_))
    print(entities.text, entities.label_)

  people = []
  for n in named_entities:
    if(n[1]=="PERSON" and " " in n[0] and not any(i.isdigit() for i in n[0])):
      people.append(n[0])

  people_extended = []
  for p in range(len(people)):
    temp = []
    temp.append(people[p])
    temp = temp+people[p].split()
    people_extended.append(temp)
    
  return people, people_extended